# Введение в глубинное обучение, ФКН ВШЭ

# Практическое задание 2. Рекуррентные Нейронные Сети

## Общая информация
Дата выдачи: 04.02.2021

Мягкий дедлайн: 19.02.2021 00:59 MSK

Жёсткий дедлайн: 22.02.2021 00:59 MSK

## Оценивание и штрафы

Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).


## Формат сдачи
Задания сдаются семинаристу на почту. Посылка должна содержать:
* Ноутбук homework-practice-02-Username.ipynb

Username — ваша фамилия на латинице

## 0. Подготовка данных (0 баллов)

Данные представляют собой корпус текстов с 4-мя категориями. Ваша задача - написать классификатор для этих данных, определяющий, к какой из категорий относится текст.

In [10]:
import numpy as np 
import string
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.layers import Dropout
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize

In [12]:
df_train = pd.read_csv('train.csv', engine=None)
df_test = pd.read_csv('test.csv')

## 1. Предобработка данных (2 балла)

В этом задании вам предстоит провести предобработку данных. Баллы ставятся следующим образом:

* Привести все тексты к одной длине, заменить слова/токены на числа, факторизовать целевую переменную и т.д. (1 балл)

* Использовать токенизатор, который разбил бы все слова на токены (подробнее https://github.com/huggingface/tokenizers). (1 балл)

### Разведовательный анализ данных
Исследуем наш target

In [13]:
df_train.groupby('source').count()

,text
source,
mchsgov,2570
mil,2562
mospolice,2427
russianpost,2235


Факторизуем целевую переменную

In [14]:
factorize_dic = {'mchsgov' : 0, 'mil' : 1, 'mospolice' : 2, 'russianpost' : 3}
unfactorize_dic = {0 : 'mchsgov', 1 : 'mil', 2 : 'mospolice', 3 : 'russianpost'}

def replace_target(df, dic):
  df = df.fillna('')
  df['source'] = df_train['source'].replace(dic)
  return df

In [15]:
def preprocess_target(df, dic): 
  df = replace_target(df, dic)
  return pd.get_dummies(df['source']).values

In [16]:
#https://github.com/susanli2016/NLP-with-Python/blob/master/Multi-Class%20Text%20Classification%20LSTM%20Consumer%20complaints.ipynb
#https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17
#Спасибо Susan Li

In [17]:
exclude = set(string.punctuation)
exclude.add("«")
exclude.add("»")
exclude.add("b")
exclude.add("r")

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('russian'))
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^а-я #+_]')

def preproces_text(text):
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    text = ''.join(ch for ch in text if ch not in exclude)
    text = re.sub(' +', ' ', text)
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 2000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

def tokine_data(df):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
    tokenizer.fit_on_texts(df['text'].values)
    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))
    return tokenizer

In [20]:
def preproces_data(df):
    df = df.fillna("")
    df['text'] = df['text'].apply(preproces_text)
    tokenizer = tokine_data(df)
    X = tokenizer.texts_to_sequences(df['text'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
return X

## 2. LSTM-сеть (4 балла)
]
В этом задании вам предстоит написать LSTM сеть __вручную__ (то есть без использования стандартных реализаций из keras / torch / tensorflow). Сама архитектура отлично расписана здесь: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

In [21]:
# Your code here ╰( ͡° ͜ʖ ͡° )つ──☆*:

## 3. Модель (2 балла)

В этом задании вам предстоит объединить вашу сеть с несколькими другими слоями для создания итоговой модели классификатора (можно начать с самой базовой архитектуры, слой эмбеддингов - LSTM - выходной слой)

In [26]:
#https://towardsdatascience.com/multi-class-text-classification-with-lstm-1590bee1bd17
#Спасибо Susan Li


model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary)

<bound method Model.summary of <tensorflow.python.keras.engine.sequential.Sequential object at 0x7fd903e4a4d0>>


## 4. Обучение модели (4 балла)

**Важно!** Public Leaderboard содержит только 30% тестовых данных, баллы за задание будут измеряться в соответствии со всеми тестовыми данными, так что после завершения соревнования Ваша позиция на leaderboard-е может поменяться.

**Важно!** При использовании **СВОЕЙ** модели LSTM полученные баллы удваиваются.

* val_accuracy > baseline: 1 балл

* Ваша позиция на лидерборде между 25% (включительно) и 50% (невключительно) квантилями среди студентов, перебивших baseline: 2 балла

* Ваша позиция на лидерборде между 50% и 75% квантилями среди студентов, перебивших baseline: 3 балла

* Ваша позиция на лидерборде между 75% и 90% квантилями среди студентов, перебивших baseline: 4 балла

* Ваша позиция на лидерборде больше 90% квантиля среди студентов, перебивших baseline: 5 баллов

* Вы в топ-3 студентов на лидерборде: 10 баллов


**Максимальный суммарный балл** - 15

**Финальная оценка**: min(суммарный балл, 10), доп баллы можно использовать для увеличения оценок за самостоятельные работы (1 доп балл = +1 балл к самостоятельной)


In [27]:
epochs = 3
batch_size = 128

history = model.fit(preproces_data(df_train), preprocess_target(df_train, factorize_dic) , 
                    epochs=epochs, 
                    batch_size=batch_size
                    )

Found 76190 unique tokens.
Epoch 1/3
78/78 [==============================] - 72s 879ms/step - loss: 1.0690 - accuracy: 0.5650
Epoch 2/3
78/78 [==============================] - 67s 864ms/step - loss: 0.2021 - accuracy: 0.9468
Epoch 3/3
78/78 [==============================] - 69s 880ms/step - loss: 0.1472 - accuracy: 0.9602


In [28]:
answer_df = pd.DataFrame(np.argmax(model.predict(preproces_data(df_test)), axis=1)).replace(unfactorize_dic)

Found 29057 unique tokens.


In [29]:

answer_df = (answer_df.reset_index()).rename(columns={'index' : 'Id', 0 : 'Category'})

In [30]:
answer_df.to_csv('answer.csv', index=False)